#  Dynamics Difference - Finite-Size Scaling 

This notebook we investigate dynamics difference (Glauber vs. Metropolis) and 
do a finite scale analysis of ICEg game. 

`Nelder-Mead'`

```
FSS Glauber  a, b, c, d, A, B
(-0.35180209040789423,
 0.0074211857718938165,
 -0.3598488333639127,
 0.002417160162042197,
 0.9392054273498407,
 0.14366396493459144)
 ```
 ```
 FSS Metropolis
 (1.4493617383153161,
 -2.080563833123093,
 -1.0733973862854667,
 0.13759713546307983,
 0.06502713383155656,
 1.0096669536678697)

```

In [1]:
#
import ising_conway
import scipy
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import dill
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.constrained_layout.use'] = True
matplotlib.use('agg')

In [2]:
data = dill.load(open("data_entropy_production.dill", "rb"))
params = data['sim_params']
sim_results = data['sim_results']

In [3]:
def get_entropy_production_df(N, M, dynamics):
    betas, entropy_production, entropy_production_se = (
        ising_conway.get_entropy_production_from_sim_results(sim_results, N, M, dynamics)
    )
    Ns = np.repeat(N, len(betas))
    Ms = np.repeat(M, len(betas))
    dynamicss = np.repeat(dynamics, len(betas))
    df = pd.DataFrame(
        {
            "N": Ns,
            "M": Ms,
            "beta": betas,
            "dynamics": dynamicss,
            "entropy_production": entropy_production,
        }
    )
    return df

In [4]:
# Assemble
# Entropy
# N, M, betas, dynamics, entropy_production
dynamics = "glauber"; 
N = 50;M = 10;
df = get_entropy_production_df(N, M, dynamics)
N = 40;M = 10;
df = pd.concat([df, get_entropy_production_df(N, M, dynamics)])
N = 30;M = 10;
df = pd.concat([df, get_entropy_production_df(N, M, dynamics)])
df_glauber = df
dynamics = "metropolis";
N = 50;M = 10;
df = get_entropy_production_df(N, M, dynamics)
N = 40;M = 10;
df = pd.concat([df, get_entropy_production_df(N, M, dynamics)])
N = 30;M = 10;
df = pd.concat([df, get_entropy_production_df(N, M, dynamics)])
df_metropolis = df

In [5]:
# KstestResult(statistic=0.5, # KS-distance
# pvalue=0.00432085032746239, 
#  statistic_location=1.1042826263381103, statistic_sign=1)
ks2 = scipy.stats.ks_2samp(df_metropolis['entropy_production'], df_glauber['entropy_production'])
ks2.statistic, ks2.pvalue

(0.5, 0.00432085032746239)

In [6]:
# Sort the data and compute ECDF (Empirical Cumulative Distribution Function)
v1 = df_metropolis['entropy_production'].sort_values()
v2 = df_glauber['entropy_production'].sort_values()

# Compute ECDF values (proportion of data ≤ value)
y1 = np.arange(1, len(v1) + 1) / len(v1)
y2 = np.arange(1, len(v2) + 1) / len(v2)

# Plot the ECDFs
font = {"family": "monospace", "weight": "bold", "size": 16}
plt.rc("font", **font)
plt.plot(v1, y1, label='Metropolis', color='blue', linewidth=2, marker='.', markersize=2)
plt.plot(v2, y2, label='Glauber', color='red', linewidth=2, marker='.', markersize=2)

# Customize the plot
plt.title('Empirical CDF Metropolis and Glauber Dynamics All N-M \n D=0.5 p-value=0.0043')
plt.xlabel('Entropy Production')
plt.ylabel('Cumulative Probability')
plt.legend()

# Show the plot
plt.show()
plt.savefig(
    f"entropy_production_cdf.png", format="png", bbox_inches="tight"
)
plt.close()


/var/folders/zs/j5jdxlz93md1tsh_thh51sm80000gn/T/ipykernel_61530/3462932160.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
df = pd.concat([df_glauber, df_metropolis])

$u(\beta) = \beta N^a M^b$  scaling variable, $u$ vs $f(u)$ should demonstrate a data collapse.



Entropy production $S_{p} = N^-c M ^-d f(u)$

$f(u) = A \cdot u + B$

In [8]:
def fobjective(params):
    X =np.array(df_glauber[['beta','N','M','entropy_production']])
    beta=X[0]
    N=X[1]
    M=X[2]
    a = params[0]
    b = params[1]
    c = params[2]
    d = params[3]
    A = params[4]
    B = params[5]
    u = beta*(N**a)*(M**b)
    f_hat = N**(-c)*M**(-d)*(A*u+B)
    f = X[3] # this is the entropy production observed
    loss = np.sum((f_hat - f) ** 2)
    return loss

initial_guess = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

result = minimize(
    fobjective,
    initial_guess,
    method='Nelder-Mead',
    options={'disp': True}  # show progress
)

a, b, c, d, A, B = tuple(result['x'])

def f_u(beta, N, M):
    u = beta*(N**a)*(M**b)
    return (A*u+B), u

fu_, u_ = f_u(df_glauber['beta'], df_glauber['N'], df_glauber['M'])
df_glauber['fu'] = fu_
df_glauber['u'] = u_
    
df_glauber['NM'] = df_glauber['N'].astype('str') + '-' + df_glauber['M'].astype('str')
df5010 = df_glauber[df_glauber['NM'] == '50-10']
df4010 = df_glauber[df_glauber['NM'] == '40-10']
df3010 = df_glauber[df_glauber['NM'] == '30-10']

plt.close()
# Customize the plot
font = {"family": "monospace", "weight": "bold", "size": 16}
plt.rc("font", **font)
plt.title('Finite Size Scaling: \n ICEg Glauber Dynamics',fontsize=16)
plt.xlabel('u scaling variable: $u(\\beta) = \\beta N^{a} M^{b}$',fontsize=16)
plt.ylabel('Scaling function: $f(u) = A \cdot u + B$',fontsize=16)


plt.plot(np.array(df5010['u']), np.array(df5010['fu']), 'o', markersize=9, label="Game size N=50 M=10")
plt.plot(np.array(df4010['u']), np.array(df4010['fu']), 'x', markersize=9, label="Game size N=40 M=10")
plt.plot(np.array(df3010['u']), np.array(df3010['fu']), '*', markersize=9, label="Game size N=30 M=10")
plt.legend()
plt.show
plt.savefig(
    f"entropy_production_finite_size_glauber.png", format="png", bbox_inches="tight"
)
plt.close()

Optimization terminated successfully.
         Current function value: 0.774566
         Iterations: 298
         Function evaluations: 485


In [9]:
a, b, c, d, A, B = tuple(result['x'])
a, b, c, d, A, B

(-0.35180209040789423,
 0.0074211857718938165,
 -0.3598488333639127,
 0.002417160162042197,
 0.9392054273498407,
 0.14366396493459144)

In [10]:
def fobjective(params):
    X =np.array(df_metropolis[['beta','N','M','entropy_production']])
    beta=X[0]
    N=X[1]
    M=X[2]
    a = params[0]
    b = params[1]
    c = params[2]
    d = params[3]
    A = params[4]
    B = params[5]
    u = beta*(N**a)*(M**b)
    f_hat = N**(-c)*M**(-d)*(A*u+B)
    f = X[3] # this is the entropy production observed
    loss = np.sum((f_hat - f) ** 2)
    return loss

initial_guess = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

result = minimize(
    fobjective,
    initial_guess,
    method='Nelder-Mead',
    options={'disp': True}  # show progress
)
a, b, c, d, A, B = tuple(result['x'])

def f_u(beta, N, M):
    u = beta*(N**a)*(M**b)
    return (A*u+B), u

fu_, u_ = f_u(df_metropolis['beta'], df_metropolis['N'], df_metropolis['M'])
df_metropolis['fu'] = fu_
df_metropolis['u'] = u_
    
df_metropolis['NM'] = df_metropolis['N'].astype('str') + '-' + df_metropolis['M'].astype('str')
df5010 = df_metropolis[df_metropolis['NM'] == '50-10']
df4010 = df_metropolis[df_metropolis['NM'] == '40-10']
df3010 = df_metropolis[df_metropolis['NM'] == '30-10']

plt.close()
# Customize the plot
font = {"family": "monospace", "weight": "bold", "size": 16}
plt.rc("font", **font)
plt.title('Finite Size Scaling: \n ICEg Metropolis Dynamics',fontsize=16)
plt.xlabel('u scaling variable: $u(\\beta) = \\beta N^{a} M^{b}$',fontsize=16)
plt.ylabel('Scaling function: $f(u) = A \cdot u + B$',fontsize=16)


plt.plot(np.array(df5010['u']), np.array(df5010['fu']), 'o', markersize=9, label="Game size N=50 M=10")
plt.plot(np.array(df4010['u']), np.array(df4010['fu']), 'x', markersize=9, label="Game size N=40 M=10")
plt.plot(np.array(df3010['u']), np.array(df3010['fu']), '*', markersize=9, label="Game size N=30 M=10")
plt.legend()
plt.show
plt.savefig(
    f"entropy_production_finite_size_metropolis.png", format="png", bbox_inches="tight"
)
plt.close()

/var/folders/zs/j5jdxlz93md1tsh_thh51sm80000gn/T/ipykernel_61530/2496739850.py:20: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  result = minimize(


In [11]:
a, b, c, d, A, B = tuple(result['x'])
a, b, c, d, A, B 

(1.4493617383153161,
 -2.080563833123093,
 -1.0733973862854667,
 0.13759713546307983,
 0.06502713383155656,
 1.0096669536678697)